# Rossmann Store Sales Prediction Example
In this example, we'll illustrate how to use NVTabular to preprocess and load tabular data for training neural networks into TensorFlow. This usees a [dataset built by FastAI](https://github.com/fastai/fastai/blob/master/courses/dl1/lesson3-rossman.ipynb) for solving the [Kaggle Rossmann Store Sales competition](https://www.kaggle.com/c/rossmann-store-sales). To expedite this tutorial, we've already lightly preprocessed the data we'll be using. For a full version of this specific example, please visit the [NVTabular GitHub](https://github.com/NVIDIA/NVTabular).

We won't go into full details of the Rossmann Kaggle competition, but below is a brief description of the task taken directly from Kaggle:

<blockquote>Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.</blockquote>

Essentially, our goal is to use features in this dataset to predict sales. There are a number of different ways to do this. You could employ a random forest classifier or even a Naive Bayes model. And in practice, it's good to try a variety of models and cross validate. However, for the purposes of this tutorial, we want to illustrate how easy it is to use data loaders and features built into NVTabular to create a deep learning model with tabular data.

Here we go!

## Do we have a GPU?

It's always a good idea to check. Really hope we do, otherwise this is going to be a lightening fast tutorial.

In [ ]:
!nvidia-smi

And of course we have the necessary imports. We'll primarily be using NVTabular, cuDF, and TensorFlow (which we'll import a bit later).

In [ ]:
import nvtabular as nvt
import os
import glob
import cudf
import s3fs

## Preparing our Dataset
Let's start by defining some of the _a priori_ information about our data, including its schema (what columns to use and what sorts of variables they represent), as well as the location of the files corresponding to some particular sampling from this schema. Note that throughout, I'll use UPPERCASE variables to represent this sort of a priori information that you might usually encode using commandline arguments or config files.

For ease of this tutorial, we've already lightly preprocessed the input data and generated nice, clean CSV files for you. You know, just like in the real world.

In [ ]:
DATA_DIR = os.environ.get('DATA_DIR', './data')

In [ ]:
# create data directory if it doesn't exist
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# configure paths for models
S3_BASE_PATH = "rapidsai-data/cyber/kdd2020/nvt/"

TRAIN_FILE = "train.csv"
VALIDATE_FILE = "valid.csv"
TEST_FILE = "test.csv"

# download the training CSV if it doesn't exist
if not os.path.exists(DATA_DIR + "/" + TRAIN_FILE):
    fs = s3fs.S3FileSystem(anon=True)
    print(">> '" + TRAIN_FILE + "' was not found, downloading now")
    fs.get(S3_BASE_PATH + TRAIN_FILE, DATA_DIR + "/" + TRAIN_FILE)
else:
    print(">> '" + TRAIN_FILE + "' was found at: " + DATA_DIR + "/" + TRAIN_FILE)
    
# download the validation CSV if it doesn't exist
if not os.path.exists(DATA_DIR + "/" + VALIDATE_FILE):
    fs = s3fs.S3FileSystem(anon=True)
    print(">> '" + TRAIN_FILE + "' was not found, downloading now")
    fs.get(S3_BASE_PATH + VALIDATE_FILE, DATA_DIR + "/" + VALIDATE_FILE)
else:
    print(">> '" + VALIDATE_FILE + "' was found at: " + DATA_DIR + "/" + VALIDATE_FILE)
    
# download the test CSV if it doesn't exist
if not os.path.exists(DATA_DIR + "/" + TEST_FILE):
    fs = s3fs.S3FileSystem(anon=True)
    print(">> '" + TEST_FILE + "' was not found, downloading now")
    fs.get(S3_BASE_PATH + TEST_FILE, DATA_DIR + "/" + TEST_FILE)
else:
    print(">> '" + TEST_FILE + "' was found at: " + DATA_DIR + "/" + TEST_FILE)

What files are available to train on in our data directory?

In [ ]:
! ls $DATA_DIR

`train.csv` and `valid.csv` seem like good candidates, let's use those.

In [ ]:
TRAIN_PATH = os.path.join(DATA_DIR, 'train.csv')
VALID_PATH = os.path.join(DATA_DIR, 'valid.csv')

### Data Exploration

Before we set about modeling, we can explore the data using cuDF. Let's just read in the training data.

In [ ]:
train_df = cudf.read_csv(TRAIN_PATH, sep=',')

In [ ]:
train_df.head()

The data is fairly wide, so we can select just one record and look at what the typical data is. cuDF doesn't support non-numeric types in the `values` call yet, but it's easy to take a small amount of data to Pandas to accomplish this.

In [ ]:
train_df.loc[0:0].to_pandas().values

We can inspect the data types of the data as well.

In [ ]:
train_df.columns.to_series().groupby(train_df.dtypes).groups

By repeating this process, we can assign columns into variables that link common data types. We're looking to predict `Sales`, so we'll denote that as our label.

In [ ]:
CATEGORICAL_COLUMNS = [
    'Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw'
]

CONTINUOUS_COLUMNS = [
    'CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday'
]
LABEL_COLUMNS = ['Sales']

COLUMNS = CATEGORICAL_COLUMNS + CONTINUOUS_COLUMNS + LABEL_COLUMNS

### Workflows and Preprocessing
A `Workflow` is used to represent the chains of feature engineering and preprocessing operations performed on a dataset, and is instantiated with a description of the dataset's schema so that it can keep track of how columns transform with each operation.

_NOTE: As of this tutorial, NVT doesn't support transforming label columns. We'll pretend it's a regular continuous column during our feature engineering phase._

In [ ]:
proc = nvt.Workflow(
    cat_names=CATEGORICAL_COLUMNS,
    cont_names=CONTINUOUS_COLUMNS+LABEL_COLUMNS,
    label_name=LABEL_COLUMNS
)

### Adding Operations to our Workflow
We add operations to a `Workflow` by leveraging the `add_(cat|cont)_feature` and `add_(cat|cont)_preprocess` methods for categorical and continuous variables, respectively. When we're done adding ops, we call the `finalize` method to let the `Workflow` build  a representation of its outputs. We use these operations to fill missing values, standardize the `Sales` column around 0 with a standard deviation of 1 (`LogOp`), normalize continuous columns, and transform categorical features into unique integer values (`Categorify`). Complete details about these functions are available on the [NVTabular's API documention site](https://nvidia.github.io/NVTabular/index.html).

In [ ]:
proc.add_cont_feature(nvt.ops.FillMissing())
proc.add_cont_preprocess(nvt.ops.LogOp(columns=['Sales']))
proc.add_cont_preprocess(nvt.ops.Normalize())
proc.add_cat_preprocess(nvt.ops.Categorify())
proc.finalize()

### Datasets
In general, the `Ops` in our `Workflow` will require measurements of statistical properties of our data in order to be leveraged. For example, the `Normalize` op requires measurements of the dataset mean and standard deviation, and the `Categorify` op requires an accounting of all the categories a particular feature can manifest. However, we frequently need to measure these properties across datasets which are too large to fit into GPU memory (or CPU memory for that matter) at once.

NVTabular solves this by providing the `dataset` object, an iterator over manageable chunks of sets of parquet or csv files that can we can use to compute statistics in an online fashion (and, later, to train neural networks in batches loaded from disk). The size of those chunks will be determined by the `gpu_memory_frac` kwarg, which will load chunks whose memory footprint is equal to that fraction of available GPU memory.

Larger chunks will lead to shorter run times due to the parallel-processing power of GPUs, but will constrain your memory and possibly lead to disk caching by expensive operations, thereby lowering efficiency.

In [ ]:
GPU_MEMORY_FRAC = 0.2
train_ds_iterator = nvt.Dataset(TRAIN_PATH, gpu_memory_frac=GPU_MEMORY_FRAC, columns=COLUMNS)
valid_ds_iterator = nvt.Dataset(VALID_PATH, gpu_memory_frac=GPU_MEMORY_FRAC, columns=COLUMNS)

Now that we have our datasets, we'll apply our `Workflow` to them and save the results out to parquet files for fast reading at train time. We'll also measure and record statistics on our training set using the `record_stats=True` kwarg so that our `Workflow` can use them at apply time.

In [ ]:
PREPROCESS_DIR = os.path.join(DATA_DIR, 'jp_ross')
PREPROCESS_DIR_TRAIN = os.path.join(PREPROCESS_DIR, 'train')
PREPROCESS_DIR_VALID = os.path.join(PREPROCESS_DIR, 'valid')
! mkdir -p $PREPROCESS_DIR_TRAIN
! mkdir -p $PREPROCESS_DIR_VALID

In [ ]:
proc.apply(train_ds_iterator, apply_offline=True, record_stats=True, output_path=PREPROCESS_DIR_TRAIN, shuffle=False)
proc.apply(valid_ds_iterator, apply_offline=True, record_stats=False, output_path=PREPROCESS_DIR_VALID, shuffle=False)

### Finalize Columns
The workflow will leverage the `Workflow.ds_to_tensors` method, which will map a dataset to its corresponding tensors. In order to make sure it runs correctly, we'll call the `create_final_cols` method to let the `Workflow` know to build the output dataset schema, and then we'll be sure to remove instances of the label column that got added to that schema when we performed processing on it.

In [ ]:
proc.create_final_cols()
# using log op and normalize on sales column causes it to get added to
# continuous columns_ctx, so we'll remove it here
while True:
    try:
        proc.columns_ctx['final']['cols']['continuous'].remove(LABEL_COLUMNS[0])
    except ValueError:
        break

## Training a Network
Now that our data is preprocessed and saved out, we can leverage `dataset`s to read through the preprocessed parquet files in an online fashion to train neural networks! Even better, using the `dlpack` library, we can pass data loaded by cuDF's accelerated parquet reader to networks in TensorFlow.

We'll start by setting some universal hyperparameters for our model and optimizer (without making any claims on the quality of these hyperparmeter choices). We leave it as an exercise to the attendee to experiment with the hyperparemeters.

In [ ]:
BATCH_SIZE = 65536
LEARNING_RATE = 1e-3
EMBEDDING_DROPOUT_RATE = 0.04
DROPOUT_RATES = [0.001, 0.01]
HIDDEN_DIMS = [1000, 500]
EPOCHS = 10

# our categorical encoder provides a handy utility for coming up with default embedding sizes
# based on the number of potential categories, so we'll just use those defaults
EMBEDDING_TABLE_SHAPES = {
    column: shape for column, shape in
        nvt.ops.get_embedding_sizes(proc).items()
}

TRAIN_PATHS = sorted(glob.glob(os.path.join(PREPROCESS_DIR_TRAIN, '*.parquet')))
VALID_PATHS = sorted(glob.glob(os.path.join(PREPROCESS_DIR_VALID, '*.parquet')))

## Data Loaders
The first thing we need to do is set up the objects for getting data into our models

`KerasSequenceDataset` wraps a lightweight iterator around a `dataset` object to handle chunking, shuffling, and application of any workflows (which can be applied online as a preprocessing step). For column names, can use either a list of string names or a list of TensorFlow `feature_columns` that will be used to feed the network

In [ ]:
import tensorflow as tf

# we can control how much memory to give tensorflow with this environment variable
# IMPORTANT: make sure you do this before you initialize TF's runtime, otherwise
# it's too late and TF will have claimed all free GPU memory
os.environ['TF_MEMORY_ALLOCATION'] = "8192" # explicit MB
os.environ['TF_MEMORY_ALLOCATION'] = "0.5" # fraction of free memory
from nvtabular.tf_dataloader import KerasSequenceDataset

# cheap wrapper to keep things some semblance of neat
def make_categorical_embedding_column(name, dictionary_size, embedding_dim):
    return tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_identity(name, dictionary_size),
        embedding_dim
    )

# instantiate our columns
categorical_columns = [
    make_categorical_embedding_column(name, *EMBEDDING_TABLE_SHAPES[name]) for
        name in CATEGORICAL_COLUMNS
]
continuous_columns = [
    tf.feature_column.numeric_column(name, (1,)) for name in CONTINUOUS_COLUMNS
]

# feed them to our datasets
train_dataset_tf = KerasSequenceDataset(
    TRAIN_PATHS, # you could also use a glob pattern
    categorical_columns+continuous_columns,
    batch_size=BATCH_SIZE,
    label_name=LABEL_COLUMNS[0],
    shuffle=True,
    buffer_size=48 # how many batches to load at once
)
valid_dataset_tf = KerasSequenceDataset(
    VALID_PATHS, # you could also use a glob pattern
    categorical_columns+continuous_columns,
    batch_size=BATCH_SIZE*4,
    label_name=LABEL_COLUMNS[0],
    shuffle=False,
    buffer_size=12
)

## Defining a Model
Next we'll need to define the inputs that will feed our model and build an architecture on top of them. For now, we'll just stick to a simple MLP model.

Using Keras, we can define the layers of our model and their parameters explicitly. Here, for the sake of consistency, I've tried to recreate the model created by FastAI as faithfully as I can given their description [here](https://docs.fast.ai/tabular.models.html#TabularModel), without making any claims as to whether this is the _right_ model to use.

In [ ]:
# DenseFeatures layer needs a dictionary of {feature_name: input}
categorical_inputs = {}
for column_name in CATEGORICAL_COLUMNS:
    categorical_inputs[column_name] = tf.keras.Input(name=column_name, shape=(1,), dtype=tf.int64)
categorical_embedding_layer = tf.keras.layers.DenseFeatures(categorical_columns)
categorical_x = categorical_embedding_layer(categorical_inputs)
categorical_x = tf.keras.layers.Dropout(EMBEDDING_DROPOUT_RATE)(categorical_x)

# Just concatenating continuous, so can use a list
continuous_inputs = []
for column_name in CONTINUOUS_COLUMNS:
    continuous_inputs.append(tf.keras.Input(name=column_name, shape=(1,), dtype=tf.float32))
continuous_embedding_layer = tf.keras.layers.Concatenate(axis=1)
continuous_x = continuous_embedding_layer(continuous_inputs)
continuous_x = tf.keras.layers.BatchNormalization(epsilon=1e-5, momentum=0.1)(continuous_x)

# concatenate and build MLP
x = tf.keras.layers.Concatenate(axis=1)([categorical_x, continuous_x])
for dim, dropout_rate in zip(HIDDEN_DIMS, DROPOUT_RATES):
    x = tf.keras.layers.Dense(dim, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization(epsilon=1e-5, momentum=0.1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
x = tf.keras.layers.Dense(1, activation='linear')(x)

# combine all our inputs into a single list
# (note that you can still use .fit, .predict, etc. on a dict
# that maps input tensor names to input values)
inputs = list(categorical_inputs.values()) + continuous_inputs
tf_model = tf.keras.Model(inputs=inputs, outputs=x)

## Define Optimizer and Train
This is probably the most conceptually consistent part between the frameworks: we'll define an objective and a method for optimizing it, then fit our model to our dataset iterators using that optimization scheme. We'll build a quick implementation of the metric Kaggle used in the original competition so that we can keep tabs on it during training.

Submissions to the Rossmann Store Sales Kaggle competition were evaulated using Root Mean Square Percentage Error (RMSPE).

$$\textrm{RMSPE}=\sqrt{\frac{1}{n}\sum_{i=1}^{n}\left ( \frac{y_i-\hat{y_i}}{y_i} \right )^2}$$

Note that we're making an explicit choice to drop zeroes to maintain consistency with Kaggle.

In [ ]:
def rmspe_tf(y_true, y_pred):
    # map back into "true" space by undoing transform
    y_true = y_true*proc.stats['stds']['Sales'] + proc.stats['means']['Sales']
    y_pred = y_pred*proc.stats['stds']['Sales'] + proc.stats['means']['Sales']

    # and then the log(1+x)
    y_true = tf.exp(y_true) - 1
    y_pred = tf.exp(y_pred) - 1

    # drop zeroes for stability (and consistency with Kaggle)
    where = tf.not_equal(y_true, 0.)
    y_true = y_true[where]
    y_pred = y_pred[where]

    percent_error = (y_true - y_pred) / y_true
    return tf.sqrt(tf.reduce_mean(percent_error**2))

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
tf_model.compile(optimizer, 'mse', metrics=[rmspe_tf])
history = tf_model.fit(
    train_dataset_tf,
    validation_data=valid_dataset_tf,
    epochs=EPOCHS
)

This does fairly well straight away, with minimal tuning of hyperparemeters and thought given to specific features and feature engineering. We also could reconsider the network structure, opting for something other than a simple MPE. In reality, it would take a RMSPE <= 0.10021 to beat the eventual winner of this specific Kaggle compeition. So while this submission won't win that Kaggle competition, the hope was to illustrate how easy it is to process data using NVTabular and feed it to a neural network in TensorFlow.

### Acknowledgments

This notebook was adapted for use in this tutorial from the [NVTabular repository](https://github.com/NVIDIA/NVTabular). It was originally created by Even Oldridge, Julio Perez, and Alec Gunny.